In [1]:
# 🧠 Imports
import pandas as pd
import os
import folium
import re
from folium.plugins import MarkerCluster

# File paths
RAW_PATH = '../data/raw/food_inspections_filtered.csv'  # ← filtered export from Chicago Data Portal
CLEAN_PATH = '../data/cleaned/food_inspections_cleaned.csv'
MAP_PATH = '../docs/grocery_stores_chicago_map.html'

# Load dataset
df = pd.read_csv(RAW_PATH)
print(f"✅ Loaded portal export: {df.shape[0]} rows")

# Keep only relevant columns
columns_to_keep = [
    'DBA Name', 'Address', 'City', 'State', 'Zip',
    'Inspection Date', 'Results', 'Violations',
    'Latitude', 'Longitude'
]
df = df[columns_to_keep]

# Drop rows with missing or invalid coordinates
df = df.dropna(subset=['Latitude', 'Longitude'])
df = df[(df['Latitude'] != '') & (df['Longitude'] != '')]

# Normalize key fields
df['DBA Name'] = df['DBA Name'].str.title().str.strip()
df['Address'] = df['Address'].str.title().str.strip()
df['Results'] = df['Results'].str.title().str.strip()

# Remove any leftover rows missing name/address
df = df[df['Address'].notnull() & df['DBA Name'].notnull()]

# Normalize for pattern matching
df['DBA_NAME_CLEAN'] = (
    df['DBA Name']
    .str.upper()
    .str.replace(r'[^A-Z0-9\s]', '', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)

df['ADDRESS_CLEAN'] = (
    df['Address']
    .str.upper()
    .str.replace(r'[^A-Z0-9\s]', '', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)

# Parse inspection dates
df['Inspection Date'] = pd.to_datetime(df['Inspection Date'], errors='coerce')

# Deduplicate: keep only most recent inspection per address + zip
df = df.sort_values(by='Inspection Date', ascending=False)
df_deduped = df.drop_duplicates(subset=['ADDRESS_CLEAN', 'Zip']).reset_index(drop=True)
print(f"📉 Deduplicated to: {df_deduped.shape[0]} unique locations")

# Define grocery and junk keyword lists
grocery_keywords = [
    'ALDI', 'JEWEL', 'MARIANO', 'WHOLE FOODS', 'FOOD MARKET',
    'SUPERMARKET', 'TRADER JOES', 'CERMARK', 'FRESH MARKET',
    'LOS SAUCES', 'CERMAK', 'BUFFALO FOODS', 'SUPER LEON',
    'AL EMAAN', 'TONYS', 'PETES', 'AMAZON GO', 'BOCKWINKELS', 'FOOD 4 LESS MIDWEST',

    # Flag real unique grocery stores
    'DELOBIAN', '5826 DIVISION FOOD', 'AFRICAN', 'AUSTIN PRODUCE',
    'AVENUE FOODS', 'AVENUE GROCERY MART', 'BROOKLYN MART', 'BROKLYN DELI MART',
    'BY THE HAND', 'BEST FOOD', 'CHICAGO AVE FOOD AND DELI',
    'CICERO CORNER FOOD', 'CICERO BIG MART', '233 MARKET', 'MARKET', 
    'CN DISCOUNTS, INC', 'F0 GROUP LLC', 'DISCOUNT STORE', 'EAGLE DISCOUNT',
    'CORCORAN GROCERY', '225 MARKET', 'AMAZON RETAIL', 'FRAMSTEAD',
    'BARGAIN FOOD', 'ANDYS', 'GROCERIES MI CUBA', 'GREENLINE FOOD',
    'HEAVEN ELEVEN', 'HUNAID FOODS', 'JIMMYS FOOD', 'JORDAN DISCOUNT'
]

junk_keywords = [
    r'7[-–\s]?ELEVEN', 'DOLLAR', 'CIRCLE K', 'SHELL', 'MOBIL',
    'GAS', 'CONVENIENCE', 'MINI MART', 'WALGREENS', 'CVS',
    'BP', 'CITGO', 'EXXON', 'AMOCO', 'LIQUOR', 'TOBACCO', 'FOOD MART', 'CORNER STORE',
    'SHOP', 'QUICK MART', 'EXPRESS', 'FAST STOP',

    # Custom junk patterns - CLEANED
    'AMSTAR', 'AMZ', 'ENTERPRISES', '24 FOODS', 'AMERICANA SUBMARINE', 'ARCO',
    'D AND D FOODMART', 'DAY NIGHT', 'A P NEWSSTAND', 'BOND DRUG COMPANY', 'CANDYALITY',
    'DESI MART', 'C B TABBACO', '76', 'BIG CORNER FOOD', 'CHICAGOS MEATERY',
    'FANNIE MAY', 'DELI MART', 'CIRCLE S MART INC.', 'EASY SNACK',
    'FOXTROT', '4 GUYS', '7 DAYS', 'A G GROCERY', 'ADAM FOOD', 'AUSTIN FOOD AND BEVERAGE',
    'NEWSTAND', 'ROCKET', 'AUGSTA', 'CRISTIS', 'DEL REY TORTILLERIA', 'CARRILLOS', 'BIRDIE',
    'FOOD, WINE', 'SPIRIT', 'CHAVA', 'CREMERIA', 'CHIVO CHIQUITA',
    'BRENDA NUTRITIONS', 'BOJORQUEZ', 'CENT', 'F.A.M.', 'CARNICERIA', 'ARAMANDOS', 'FALCON 300',
    'HARRISON STREET',

    # Additional Austin unclassified patterns
    '123 FOOD MARKET', '365 FOODMART', '4 BROTHERS FOOD MART', '5826 DIVISION FOOD',
    'ADAM & ARAM', 'AFRICAN 09', 'APPLE FOOD MART', 'AUGUSTA FOOD', 'AVENUE GROCERY MART',
    'AYMAN FOOD MARKET', 'BARGAIN FOOD', "BIG MOE'S", 'BOBOS MARKET', 'BROOKLYN DELI MART'
]


# Flag junk and grocery stores
df_deduped['IS_JUNK_STORE'] = df_deduped['DBA_NAME_CLEAN'].apply(
    lambda name: any(re.search(pattern, name) for pattern in junk_keywords)
)

df_deduped['IS_REAL_GROCERY'] = df_deduped['DBA_NAME_CLEAN'].apply(
    lambda name: any(keyword in name for keyword in grocery_keywords)
)

# Add extra signal from violations
produce_hints = ['fruit', 'vegetable', 'cut', 'salad', 'raw', 'cold', 'prep']
df_deduped['HAS_PRODUCE_FLAG'] = df_deduped['Violations'].fillna('').str.lower().apply(
    lambda v: any(word in v for word in produce_hints)
)
df_deduped['IS_REAL_GROCERY'] = df_deduped['IS_REAL_GROCERY'] | df_deduped['HAS_PRODUCE_FLAG']

# Remove conflicting labels (prioritize real grocery)
conflicts = df_deduped[df_deduped['IS_REAL_GROCERY'] & df_deduped['IS_JUNK_STORE']]
df_deduped.loc[conflicts.index, 'IS_JUNK_STORE'] = False

# Save cleaned dataset
os.makedirs(os.path.dirname(CLEAN_PATH), exist_ok=True)
df_deduped.to_csv(CLEAN_PATH, index=False)
print(f"✅ Cleaned dataset saved to: {CLEAN_PATH}")

# Build interactive map
m = folium.Map(location=[41.8781, -87.6298], zoom_start=11)
marker_cluster = MarkerCluster().add_to(m)

for _, row in df_deduped.iterrows():
    lat, lon = row['Latitude'], row['Longitude']
    if pd.isna(lat) or pd.isna(lon):
        continue

    if row['IS_REAL_GROCERY']:
        color, icon = 'green', 'shopping-cart'
    elif row['IS_JUNK_STORE']:
        color, icon = 'red', 'remove'
    else:
        color, icon = 'gray', 'info-sign'

    popup = f"{row['DBA Name']}<br>{row['Address']}<br>ZIP: {int(row['Zip']) if pd.notnull(row['Zip']) else ''}"
    folium.Marker(
        location=[lat, lon],
        popup=popup,
        icon=folium.Icon(color=color, icon=icon)
    ).add_to(marker_cluster)

# Save map to file
os.makedirs(os.path.dirname(MAP_PATH), exist_ok=True)
m.save(MAP_PATH)
print(f" Map saved to: {MAP_PATH}")

# Save labeled + deduped grocery dataset for analysis/visualization
df_deduped.to_csv('../data/cleaned/grocery_stores_cleaned_v1.csv', index=False)
print("✅ grocery_stores_cleaned_v1.csv saved!")



/Users/archangel/Desktop/chi-food-access-map/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


✅ Loaded portal export: 35681 rows
📉 Deduplicated to: 4503 unique locations
✅ Cleaned dataset saved to: ../data/cleaned/food_inspections_cleaned.csv
 Map saved to: ../maps/grocery_stores_chicago_map.html
✅ grocery_stores_cleaned_v1.csv saved!
